# Prática: Scrapping

## Introdução

Hoje veremos 4 exemplos e exercícios de Scrapping real:

| Metodologia | Caso de Uso Primário | Como Funciona | Lida com JavaScript? | Escalabilidade | Complexidade | Vantagem Chave | Desvantagem Chave |
| :--- | :--- | :--- | :--- | :--- | :--- | :--- | :--- |
| **API (Spotify/YouTube)** | Acessar dados estruturados e oficialmente fornecidos. | Requisições HTTP para endpoints JSON. | N/A (Dados pré-estruturados) | Limitada por cotas/limites de taxa. | Baixa a Moderada | Confiabilidade e Qualidade dos Dados | Campos de dados limitados e limites de taxa. |
| **Scraping Estático (Requests + BeautifulSoup)** | Raspar sites simples, renderizados no servidor. | Requisições HTTP para HTML, depois análise. | Não | Moderada | Baixa | Simplicidade e Velocidade | Falha em sites dinâmicos. |
| **Scraping Dinâmico (Playwright)** | Interagir com aplicações pesadas em JavaScript. | Automatizar um navegador real para renderizar páginas. | Sim | Baixa (uso intensivo de recursos) | Moderada a Alta | Lida com qualquer site moderno. | Lento e com uso intensivo de recursos. |
| **Crawling Assíncrono (Scrapy)** | Extração de dados em larga escala e de várias páginas. | Motor assíncrono gerenciando requisições e callbacks. | Não (por padrão) | Alta | Alta | Desempenho e Estrutura | Curva de aprendizado acentuada. |

## 0. Instalação das Bibliotecas

Antes de começar, vamos instalar todas as ferramentas necessárias. Execute a célula abaixo.

In [3]:
!pip install pandas spotipy google-api-python-client beautifulsoup4 requests playwright scrapy ipykernel
!playwright install

  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 5.6 MB/s  0:00:02 eta 0:00:01
Using cached pyparsing-3.2.3-py3-none-any.whl (111 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.7/38.7 MB 6.7 MB/s  0:00:05m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 6.6 MB/s  0:00:01 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 6.7 MB/s  0:00:01 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 7.3 MB/s  0:00:00m0:00:0100:01
Using cached setuptools-80.9.0-py3-none-any.whl (1.2 MB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 58.0.4
    Uninstalling setuptools-58.0.4:
      Successfully uninstalled setuptools-58.0.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43/43 [google-api-python-client]n-client]n-protos]
129.7 MiB [                    ] 0% 0.0s129.7 MiB [      

---
## Módulo 1: Recuperação Estruturada de Dados via APIs - Spotify

Este módulo introduz o método mais robusto e ético de aquisição de dados: o uso de uma Interface de Programação de Aplicativos (API) formal.

Utilizaremos a Web API do Spotify, por meio da biblioteca `spotipy`, como um estudo de caso em recuperação de dados estruturados.

### 1.1. Estrutura Conceitual: O "Contrato" de uma API

Utilizar uma API é análogo a assinar um contrato com um provedor de dados.

O desenvolvedor concorda com os termos do provedor (autenticação, limites de taxa) e, em troca, o provedor concorda em fornecer dados limpos e estruturados, geralmente em formato JSON.

Esta relação é fundamentalmente diferente do web scraping tradicional, que não possui tal acordo e opera analisando o conteúdo destinado à exibição humana.

* **Autenticação:** O acesso à maioria das APIs requer autenticação para identificar a aplicação solicitante. O Spotify oferece vários fluxos de autorização. Para dados que não são específicos do usuário (como playlists públicas, informações de artistas ou episódios de podcast), o **Fluxo de Credenciais do Cliente** é o mais adequado. Este método autentica a própria aplicação, em vez de um usuário, usando um `Client ID` e um `Client Secret` para obter um token de acesso.
* **Endpoints e Escopo:** Um endpoint é um URL específico que fornece acesso a um recurso de dados particular (por exemplo, `/v1/playlists/{playlist_id}` ou `/v1/shows/{id}/episodes`). O escopo de dados que uma aplicação pode acessar é determinado pelas permissões concedidas durante o processo de autorização.
* **Paginação:** Para gerenciar a carga do servidor e garantir respostas rápidas, as APIs raramente retornam conjuntos de dados inteiros em uma única requisição. Em vez disso, os dados são divididos em "páginas". O desenvolvedor deve iterar por essas páginas para coletar o conjunto de dados completo, geralmente verificando um token `next` ou gerenciando parâmetros de `offset` e `limit` na resposta da API.

### 1.2. Pré-requisitos: Obtendo Suas Credenciais da API do Spotify

Antes de fazer qualquer chamada à API, é necessário registrar uma aplicação no painel do Spotify para Desenvolvedores. Este processo é um primeiro passo crucial para qualquer aluno.

1.  **Configurar a Conta:** Faça login no([https://developer.spotify.com/dashboard](https://developer.spotify.com/dashboard)). Se solicitado, leia e aceite os Termos de Serviço do Desenvolvedor mais recentes.
2.  **Criar uma Aplicação:** No seu painel, clique no botão "Create an app". Preencha o nome e a descrição da aplicação. Para o "Redirect URI", um valor como `http://localhost:8888/callback` é suficiente para este fluxo de autenticação.
3.  **Recuperar Credenciais:** Após a criação da aplicação, navegue até as configurações dela. O `Client ID` será exibido diretamente. Clique em "View client secret" para revelar o `Client Secret`. Essas duas chaves são essenciais e devem ser mantidas em segredo, pois funcionam como o nome de usuário e a senha da sua aplicação.

A gente pode salvar a chave em um arquivo json em separado, para não ter nossa chave em texto puro no meio do código. Criem um arquivo json na mesma pasta chamado auth.json com um conteúdo tipo assim:

`{"client_id": "5e9a80618b284145b54bb1f7df94bb6c",
"client_secret": "0cdef7160e4143118e48abdd939668e8"}`

Vamos fazer um teste rápido para ver se está funcionando:

In [1]:
import json

cred = json.load(open('auth.json'))
client_id = cred['client_id']
client_secret = cred['client_secret']

### 1.3. Exemplo 1: Raspagem de Faixas de Playlist e Características de Áudio

**Objetivo:** Recuperar todas as faixas de uma playlist pública popular (por exemplo, "Today's Top Hits" do Spotify) e enriquecer esses dados com características de áudio (como dançabilidade, energia, valência) para cada faixa.

**Passo a Passo do Código:**

Este script demonstra como autenticar, lidar com a paginação para buscar todas as faixas de uma playlist, fazer requisições em lote para otimizar as chamadas à API e, finalmente, consolidar os dados em um DataFrame do pandas.

In [2]:
# 1. Configuração: Importar bibliotecas e definir credenciais
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import os

# IMPORTANTE: Substitua pelos seus valores obtidos no dashboard do Spotify
CLIENT_ID = client_id
CLIENT_SECRET = client_secret

# 2. Autenticação
try:
    auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
    sp = spotipy.Spotify(auth_manager=auth_manager)
    print("Autenticação bem-sucedida!")
except Exception as e:
    print(f"Erro na autenticação: {e}. Verifique seu Client ID e Client Secret.")




/Users/michaelwillian/Documents/Repositórios/Ci-ncia_de_dados/Meu_Ambiente/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Autenticação bem-sucedida!


In [3]:
# 3. Buscar Faixas (Lidando com Paginação)
# URI da playlist "Today's Top Hits"
playlist_uri = '44kx17AFDREGQb1QLGh48O'

# Lista para armazenar todos os itens da playlist
all_playlist_items = []

# Primeira chamada para obter o primeiro lote de faixas
results = sp.playlist_items(playlist_uri)
all_playlist_items.extend(results['items'])

# Loop para buscar as páginas restantes
while results['next']:
    results = sp.next(results)
    all_playlist_items.extend(results['items'])

# 4. Extrair IDs das Faixas
track_ids = []
for item in all_playlist_items:
    # Algumas playlists podem ter itens nulos ou que não são faixas
    if item.get('track') and item.get('track', {}).get('id'):
        track_ids.append(item['track']['id'])
print(track_ids)

['0SfcG65T1KKCj5NQffpzQR', '6kyljOCPWaPk0ISkx2Us8T', '5Fydh31AheclytAjf3ae2o', '7FpQLWSBgpNMcYrYpa8gwQ', '1r1GzEVVA4fm9XVUfgc7TR', '58EIg6uXrVWZX0ZCEL9STj', '05ysSyVvvSMqS6gxUPBOco', '51oUGjgIJb0f51dcvfAPLF', '4D3HlCklYQQgTEuPDso96G', '7MnT7msJZg3XBAS0OTfGrB', '6L5EaSYdQDLQHhaNPUlUs1', '0XBQo5SfE4nU3KdYdKBnnw', '7FHVPjQyq8fdVWMV4qEJs9', '5flYFbJweqUHlQjle7uRlI', '2r05fN1qdcN0y66l0vzUGY', '00zSdWnTqK1sQmfkV6gZBZ', '6Pa6VpdGS8OfiVOEnNAHHw', '6M9Fv1HcgALPJmPZqBv4tb', '6wpDQGn3Gl0j9Wt6D6mYvQ', '2FP3xfIkR1rUyt5EKN6YsN', '78AuK7p3Wj5a1cvIqfX5cp', '2Yk0HvfTaijA47aM0Fj88u', '6nq1UXqPzEPPIG54YMSnhn', '4N56HWFXU59sSGEgy0p0Tl', '46s6p0tsgRBfzWriIg3w9o', '624ra5mDiibqvFYDK4yhmo', '7I2hCftnkF1gI7zYOegATu', '7hkQhMFq4EOTYwX3I7cgmA', '7pn5QXr2LTeGsgHyCaJxDo', '5MwzIbc8K9a0eLbf0pivVK', '7xmAI8DoydDiXSnVaknxh7', '5shQSAr34JZE03N4YtlMIX', '0rOQyLZJ9oASgtdoDZgrmK', '7nJuEpX0qlpKXxUK0ioIMj', '5HDlh6UhT3AMQs935wE1qr', '7xpDGoUGbtaquk5xMvzwTh', '7zMKyiC6E1bdx7AMlA7NZM', '74Es0YAvJFbFCZ6ULMcydb', '2Ri9vnXqST

In [4]:



# 6. Consolidação de Dados e Criação do DataFrame
track_data = []
for item in all_playlist_items:
    if item.get('track') and item.get('track', {}).get('id'):
        track = item['track']
        track_info = {
            'track_id': track['id'],
            'track_name': track['name'],
            'artist_name': ', '.join([artist['name'] for artist in track['artists']]),
            'album_name': track['album']['name'],
            'popularity': track['popularity']
        }
        track_data.append(track_info)

# Criar DataFrames
df_tracks = pd.DataFrame(track_data)


# Renomear a coluna 'id' para corresponder e fazer o merge

# Exibir o resultado
print(f"Total de faixas coletadas: {len(df_tracks)}")
df_tracks.head()

Total de faixas coletadas: 318


,track_id,track_name,artist_name,album_name,popularity
0,0SfcG65T1KKCj5NQffpzQR,Não Quero Dinheiro (Só Quero Amar),Tim Maia,Tim Maia 1971,64
1,6kyljOCPWaPk0ISkx2Us8T,País Tropical,Jorge Ben Jor,Jorge Ben Sem Limite,31
2,5Fydh31AheclytAjf3ae2o,Meu Amigo Pedro,Raul Seixas,Há 10 Mil Anos Atrás,58
3,7FpQLWSBgpNMcYrYpa8gwQ,Onde Você Mora? (Acústico),Cidade Negra,Sobre Todas As Forças,53
4,1r1GzEVVA4fm9XVUfgc7TR,A feira,O Rappa,Rappa-Mundi,59


### 1.4. Exemplo 2: Raspagem de Episódios de um Podcast

**Objetivo:** Recuperar metadados de todos os episódios de um programa de podcast específico.

**Passo a Passo do Código:**

Este script utiliza o endpoint `show_episodes` para buscar informações sobre os episódios de um podcast, lidando novamente com a paginação para garantir que todos os episódios sejam coletados.

In [17]:
# (Reutilize a configuração de autenticação do exemplo anterior)
# Se estiver executando esta célula isoladamente, descomente as linhas abaixo
# import spotipy
# from spotipy.oauth2 import SpotifyClientCredentials
# import pandas as pd

# CLIENT_ID = "SEU_CLIENT_ID_AQUI"
# CLIENT_SECRET = "SEU_CLIENT_SECRET_AQUI"

# auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
# sp = spotipy.Spotify(auth_manager=auth_manager)

# URI do podcast "Medo e Delírio em Brasilia"
show_uri = '4GTrddwqYaFDOuNUPcsRaX'

# Lista para armazenar todos os episódios
all_episodes = []

# Primeira chamada
results = sp.show_episodes(show_uri, limit=50)
all_episodes.extend(results['items'])

# Loop de paginação
while results['next']:
    results = sp.next(results)
    all_episodes.extend(results['items'])

# Extrair dados relevantes
print(len(all_episodes))
episodes_data = []
for episode in all_episodes:
    if episode == None:
      continue
    episode_info = {
        'episode_id': episode['id'],
        'name': episode['name'],
        'description': episode['description'],
        'release_date': episode['release_date'],
        'duration_ms': episode['duration_ms'],
        'url': episode['external_urls']['spotify']
    }
    episodes_data.append(episode_info)

# Criar o DataFrame
df_episodes = pd.DataFrame(episodes_data)

# Exibir o resultado
print(f"Total de episódios coletados: {len(df_episodes)}")
df_episodes.head()

789
Total de episódios coletados: 785


,episode_id,name,description,release_date,duration_ms,url
0,3LJ90rEki97pqneK8TsEHb,II - 2025.58 - O segundo julgamento mais impor...,Judgement weeks!Adquira o “DORMINDO ENTRE CADÁ...,2025-09-06,5058606,https://open.spotify.com/episode/3LJ90rEki97pq...
1,3I92nr54KbNxfoFPIfGSLS,II - 2025.57 - Tudo dando incrivelmente errado...,Eu fico muito triste com uma notícia dessas.Pr...,2025-09-03,4364237,https://open.spotify.com/episode/3I92nr54KbNxf...
2,4cKJGsm8AvtTO7xfPwb1RI,"II - 2025.56 - Briguem, desgraçados!","“Ah, porra! Cês parecem malucos, porra!”",2025-08-30,3758660,https://open.spotify.com/episode/4cKJGsm8AvtTO...
3,3CxTUL6h2s9SOczJobhxYB,II - 2025.55 - O dia em que o Mendonça chamou ...,Terrivelmente evangélico vs. Maridão de Dona Vivi,2025-08-27,4227889,https://open.spotify.com/episode/3CxTUL6h2s9SO...
4,4jw3Ls9fWwh9vLonVhv9mN,"II - 2025.54 - ""VTNC SEU INGRATO DO C@R@LH0""","“Muita, mas muita coisa”",2025-08-23,4010550,https://open.spotify.com/episode/4jw3Ls9fWwh9v...


### 1.5. Exercício do Módulo 1

**Tarefa:** Escolha um artista musical de sua preferência e recupere sua discografia completa. O script deve:

1.  Usar `sp.artist_albums()` para obter todos os álbuns do artista (lembre-se de lidar com a paginação).
2.  Para cada álbum obtido, usar `sp.album_tracks()` para obter todas as suas faixas.
3.  Consolidar todos os dados em um único DataFrame do pandas com as colunas: `album_name`, `album_release_date`, `track_name` e `track_number`.

**Objetivo de Aprendizagem:** Este exercício reforça o conceito de paginação e demonstra como encadear múltiplas chamadas de API para construir um conjunto de dados abrangente, uma tarefa comum em projetos de análise de dados.

In [20]:
# Coloque seu código aqui
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id=client_id,
    client_secret=client_secret,
))
artist_id = "68YeXpLt3jB7JHQS5ZjMGo"

albums = []
results = sp.artist_albums(artist_id=artist_id, album_type='album', limit=50)
albums.extend(results["items"])

unique_albums = {album["id"]:album for album in albums}.values()

data = []

for album in unique_albums:
    album_name = album["name"]
    album_date = album["release_date"]

    tracks = sp.album_tracks(album_id=album['id'])
    while tracks:
        for track in tracks['items']:
            data.append({
                "album_name" :album_name,
                "album_release_date": album_date,
                "track_name": track['name'],
                "track_number": track['track_number']
            })
        if tracks['next']:
            tracks = sp.next(tracks)
        else:
            tracks = None

df = pd.DataFrame(data)
print(df)

df.to_csv("discografia.csv", index=False)




           album_name album_release_date         track_name  track_number
0   MAIOR QUE O TEMPO         2025-03-27           NÃO PARE             1
1   MAIOR QUE O TEMPO         2025-03-27          BEM NOVIN             2
2   MAIOR QUE O TEMPO         2025-03-27    PRETTY LET GIRL             3
3   MAIOR QUE O TEMPO         2025-03-27    LEITE DERRAMADO             4
4   MAIOR QUE O TEMPO         2025-03-27          YES OR NO             5
5   MAIOR QUE O TEMPO         2025-03-27           TEMPORAL             1
6   MAIOR QUE O TEMPO         2025-03-27   OQQELESVAOFALAR?             2
7   MAIOR QUE O TEMPO         2025-03-27  PREVENDO A JOGADA             3
8   MAIOR QUE O TEMPO         2025-03-27             GRÉCIA             4
9   MAIOR QUE O TEMPO         2025-03-27     MULHER SECRETA             5
10  MAIOR QUE O TEMPO         2025-03-27  MAIOR QUE O TEMPO             6


---

## Módulo 2: Navegando no Ecossistema do Google - A API de Dados do YouTube v3

Este módulo se baseia nos fundamentos de API, introduzindo o ecossistema de APIs do Google, que envolve chaves de API e um sistema de cotas, apresentando um conjunto diferente de desafios e considerações.

### 2.1. Estrutura Conceitual: Chaves de API e Gerenciamento de Cotas

* **Chaves de API vs. OAuth:** Para acessar dados públicos, a API do YouTube utiliza uma chave de API simples para identificação, em vez do fluxo OAuth mais complexo, que é necessário para dados privados do usuário. A chave de API identifica o projeto que está fazendo a requisição.
* **O Conceito de "Cota":** Diferentemente dos limites de taxa baseados em tempo do Spotify, a API do YouTube utiliza um sistema de "cotas" diárias. Cada tipo de requisição tem um "custo" diferente. Uma leitura simples de vídeo pode custar 1 unidade de cota, enquanto uma busca pode custar 100 unidades. Este sistema força os desenvolvedores a serem extremamente eficientes com suas chamadas à API. A existência de um sistema de cotas é, em si, uma estratégia de gerenciamento de recursos e monetização por parte do Google. Ao atribuir custos, eles desincentivam a raspagem ineficiente ou de alto volume e podem oferecer cotas maiores mediante pagamento. Essa compreensão influencia diretamente a estratégia de codificação, priorizando menos chamadas de API, porém mais direcionadas.
* **Parâmetro `part`:** As chamadas à API do YouTube são otimizadas especificando qual `part` de um recurso se deseja recuperar (por exemplo, `snippet`, `statistics`, `contentDetails`). Solicitar apenas as partes necessárias reduz o uso da cota e o tamanho da resposta, tornando o processo mais eficiente.

### 2.2. Pré-requisitos: Gerando uma Chave da API de Dados do YouTube

Para interagir com a API do YouTube, é necessário obter uma chave de API no Google Cloud Console.

1.  **Acessar o Console:** Navegue até a [página de Credenciais](https://console.cloud.google.com/apis/credentials) no Google API Console. Pode ser necessário criar um novo projeto se você não tiver um.
2.  **Habilitar a API:** No painel, vá para a "Biblioteca de APIs", procure por "YouTube Data API v3" e habilite-a para o seu projeto.
3.  **Criar Chave de API:** De volta à página de "Credenciais", clique em "Criar credenciais" e selecione "Chave de API".
4.  **Restringir a Chave (Recomendado):** Após a criação da chave, é altamente recomendável restringi-la para evitar o uso não autorizado. Clique em "Restringir chave" e selecione "YouTube Data API v3" na lista de APIs. Isso garante que sua chave só possa ser usada para este serviço específico.

### 2.3. Exemplo 1: Agregando Estatísticas de Vídeos

**Objetivo:** Para uma lista de IDs de vídeo, recuperar estatísticas detalhadas como contagem de visualizações, curtidas e comentários.

**Passo a Passo do Código:**

Este script utiliza o método `videos().list` para buscar dados em lote, uma prática essencial para conservar a cota da API.

In [18]:
# 1. Configuração
import pandas as pd
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# IMPORTANTE: Substitua pela sua chave de API
API_KEY = cred['youtube_key']
YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_VERSION = 'v3'

# 2. Cliente da API
try:
    youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=API_KEY)
    print("Cliente da API do YouTube criado com sucesso!")
except Exception as e:
    print(f"Erro ao criar cliente da API: {e}. Verifique sua chave de API.")

def get_video_stats(video_ids):
    """Busca estatísticas para uma lista de IDs de vídeo."""
    all_video_stats = []

    # 3. Requisições em Lote (chunks de 50)
    for i in range(0, len(video_ids), 50):
        chunk_ids = video_ids[i:i + 50]
        try:
            # 4. Chamada à API
            request = youtube.videos().list(
                part='snippet,statistics',
                id=','.join(chunk_ids)
            )
            response = request.execute()

            # 5. Extração de Dados
            for video in response['items']:
                stats = {
                    'video_id': video['id'],
                    'title': video['snippet']['title'],
                    'published_at': video['snippet']['publishedAt'],
                    'view_count': int(video['statistics'].get('viewCount', 0)),
                    'like_count': int(video['statistics'].get('likeCount', 0)),
                    'comment_count': int(video['statistics'].get('commentCount', 0))
                }
                all_video_stats.append(stats)
        except HttpError as e:
            print(f"Ocorreu um erro HTTP: {e.resp.status} {e.content}")

    return pd.DataFrame(all_video_stats)

# Lista de exemplo de IDs de vídeo (Nerdologia e outros)
video_ids = ['GJrOm8qOwIY', 'YQ_xWvX1n9g']
df_videos = get_video_stats(video_ids)

# Exibir resultado
df_videos

Cliente da API do YouTube criado com sucesso!


,video_id,title,published_at,view_count,like_count,comment_count
0,GJrOm8qOwIY,FALHA DE COBERTURA #253: Futebol Português,2025-09-09T01:04:59Z,137212,18958,825
1,YQ_xWvX1n9g,Line Goes Up – The Problem With NFTs,2022-01-21T17:00:03Z,17306414,439714,63992


### 2.4. Exemplo 2: Desconstruindo uma Playlist do YouTube

**Objetivo:** Recuperar uma lista de todos os vídeos contidos em uma playlist pública específica.

**Passo a Passo do Código:**

Este script demonstra como lidar com a paginação na API do YouTube usando o `nextPageToken` para garantir que todos os itens da playlist sejam coletados.

In [9]:
# (Reutilize a configuração do cliente da API do exemplo anterior)
def get_playlist_videos(playlist_id):
    """Busca todos os vídeos de uma playlist."""
    all_videos = []
    next_page_token = None

    # 2. Loop de Paginação
    while True:
        try:
            # 3. Chamada à API
            request = youtube.playlistItems().list(
                part='snippet',
                playlistId=playlist_id,
                maxResults=50,
                pageToken=next_page_token
            )
            response = request.execute()

            for item in response['items']:
                video_info = {
                    'video_id': item['snippet']['resourceId']['videoId'],
                    'title': item['snippet']['title'],
                    'published_at': item['snippet']['publishedAt']
                }
                all_videos.append(video_info)

            # Atualiza o token para a próxima página
            next_page_token = response.get('nextPageToken')
            if not next_page_token:
                break
        except HttpError as e:
            print(f"Ocorreu um erro HTTP: {e.resp.status} {e.content}")
            break

    # 4. Criação do DataFrame
    return pd.DataFrame(all_videos)

# ID da playlist de exemplo (Nerdologia)
playlist_id = 'PLmAZw8oyG75iM40SIzhaxKoZ8-rKT-OPw'
df_playlist = get_playlist_videos(playlist_id)

# Exibir resultado
print(f"Total de vídeos na playlist: {len(df_playlist)}")
df_playlist.head()

Total de vídeos na playlist: 63


,video_id,title,published_at
0,4ZeQ7VoWT0U,1.1 - Gerência de Processos [SO UFAM],2022-06-08T15:21:21Z
1,LyAAXCNuU6g,1.2 - Gerência de Processos [SO UFAM],2022-06-08T15:21:29Z
2,0zzro0GYhl4,1.3 - Estados de um processo [SO UFAM],2022-06-08T15:21:35Z
3,nSnc-QSHYYU,1.4 - Estrutura de um processo [SO UFAM],2022-06-08T15:21:42Z
4,OY2xlLSLjHQ,2.1 - Execução Direta Limitada [SO UFAM],2022-06-15T21:37:44Z


### 2.5. Exercício do Módulo 2

**Tarefa:** Encontre um canal do YouTube de seu interesse. Você precisará encontrar manualmente o ID da playlist de "uploads" do canal (geralmente, o ID do canal com "UU" substituindo os dois primeiros caracteres "UC"). Em seguida, escreva um script que:

1.  Use o código de raspagem de playlist para obter os IDs dos 50 vídeos mais recentes desse canal.
2.  Passe esses 50 IDs de vídeo para o código de estatísticas de vídeo para obter as contagens de visualizações, curtidas e comentários de cada um.
3.  Crie um único DataFrame com as colunas: `video_title`, `publish_date`, `view_count`, `like_count` e `comment_count`.

**Objetivo de Aprendizagem:** Este exercício combina ambos os exemplos do módulo, ensinando os alunos a encadear diferentes endpoints da API para responder a uma pergunta mais complexa e a aplicar o conhecimento de forma prática.

In [ ]:
# Coloque seu código aqui

---

## Módulo 3: A Fundação do Web Scraping - `requests` e `BeautifulSoup`

Este módulo faz a transição do mundo estruturado das APIs para a natureza não estruturada do HTML, ensinando as habilidades fundamentais do web scraping.

### 3.1. Estrutura Conceitual: A Requisição HTTP e o Documento HTML

* **Requisição/Resposta HTTP:** O modelo cliente-servidor é a base da web. A biblioteca `requests` envia uma requisição HTTP GET para um servidor, que, por sua vez, retorna o conteúdo HTML bruto da página como uma string de texto.
* **O `User-Agent`:** Os servidores da web podem identificar e, por vezes, bloquear requisições que não parecem vir de um navegador real. Por padrão, a biblioteca `requests` se identifica como um script Python. Para aumentar a confiabilidade do scraper e simular um navegador, é uma prática essencial definir um cabeçalho `User-Agent` em cada requisição.
* **Análise de HTML (Parsing):** O HTML bruto retornado é apenas texto. A biblioteca `BeautifulSoup` analisa esse texto e o transforma em uma estrutura de árvore navegável (o Document Object Model, ou DOM), permitindo que elementos específicos sejam encontrados e extraídos programaticamente.
* **Seletores CSS:** A maneira mais comum e intuitiva de localizar elementos em uma árvore HTML é através de seletores CSS. Eles fornecem uma sintaxe concisa para selecionar elementos por tag (ex: `h1`), classe (ex: `.product-title`) ou ID (ex: `#main-content`), bem como combinações mais complexas, como seletores de descendentes (`div.price`) para encontrar um elemento de preço dentro de uma `div`.

### 3.2. Exemplo: Raspagem de uma Lista de Produtos de E-commerce

**Objetivo:** Raspar informações de produtos (nome, preço, avaliação) de uma listagem de categorias com várias páginas em um site de e-commerce real, `thewhiskyexchange.com`. Este site é um exemplo mais robusto do que um sandbox, pois apresenta uma complexidade de HTML do mundo real.

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# 1. Configuração inicial
base_url = "https://www.nuuvem.com/br-pt/catalog/price/promo/sort/bestselling/sort-mode/desc"
# É crucial definir um User-Agent para simular um navegador
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36 Edge/16.16299'
}

product_links = []
# 2. Paginação: Iterar pelas 3 primeiras páginas da categoria de uísque japonês
for page_num in range(1, 4):
    # O padrão de URL para paginação foi identificado inspecionando o site
    url = f'https://www.nuuvem.com/br-pt/catalog/price/promo/sort/bestselling/sort-mode/desc/page/{page_num}'
    print(f"Processando página de listagem: {url}")

    # 3. Requisição e Análise da página de listagem
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    # 4. Extração dos links dos produtos
    # O seletor foi encontrado usando as ferramentas de desenvolvedor do navegador

    product_list = soup.find_all("div", {"class": "grid-col-6"})
    print(len(product_list))
    for product in product_list:


        link_tag = product.find("a")
        if link_tag and 'href' in link_tag.attrs:
            product_links.append(link_tag['href'])

    print(f"Página {page_num} processada. Total de links coletados: {len(product_links)}")
    time.sleep(1) # Pausa para ser um bom cidadão da web

print(product_links)

Processando página de listagem: https://www.nuuvem.com/br-pt/catalog/price/promo/sort/bestselling/sort-mode/desc/page/1
20
Página 1 processada. Total de links coletados: 20
Processando página de listagem: https://www.nuuvem.com/br-pt/catalog/price/promo/sort/bestselling/sort-mode/desc/page/2
20
Página 2 processada. Total de links coletados: 40
Processando página de listagem: https://www.nuuvem.com/br-pt/catalog/price/promo/sort/bestselling/sort-mode/desc/page/3
20
Página 3 processada. Total de links coletados: 60
['https://www.nuuvem.com/br-pt/item/silent-hill-homecoming', 'https://www.nuuvem.com/br-pt/item/borderlands-4', 'https://www.nuuvem.com/br-pt/item/castlevania-lords-of-shadow-ultimate-edition', 'https://www.nuuvem.com/br-pt/item/resident-evil-5-gold-edition', 'https://www.nuuvem.com/br-pt/item/digimon-story-time-stranger', 'https://www.nuuvem.com/br-pt/item/resident-evil-6-complete', 'https://www.nuuvem.com/br-pt/item/lords-of-the-fallen-goty', 'https://www.nuuvem.com/br-pt/it

In [6]:

# 5. Raspagem das páginas de detalhes
all_products_data = []
for link in product_links:
    print(f"Raspando detalhes de: {link}")
    response = requests.get(link, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    # 6. Extração dos dados com tratamento de exceções
    try:
        name = soup.find("h1", {"class": "product-title"}).text.strip()
    except AttributeError:
        name = None

    try:
        price = soup.find("span", {"class": "product-price--val"}).text.strip() #TA ERRADO
    except AttributeError:
        price = None



    product_data = {
        'name': name,
        'price': price,
        'url': link
    }
    all_products_data.append(product_data)
    time.sleep(1)

# 7. Criação do DataFrame
df_jogos = pd.DataFrame(all_products_data)

# Exibir resultado
print(f"\nTotal de produtos raspados: {len(df_jogos)}")
df_jogos.head()

Raspando detalhes de: https://www.nuuvem.com/br-pt/item/silent-hill-homecoming
Raspando detalhes de: https://www.nuuvem.com/br-pt/item/borderlands-4
Raspando detalhes de: https://www.nuuvem.com/br-pt/item/castlevania-lords-of-shadow-ultimate-edition
Raspando detalhes de: https://www.nuuvem.com/br-pt/item/resident-evil-5-gold-edition
Raspando detalhes de: https://www.nuuvem.com/br-pt/item/digimon-story-time-stranger
Raspando detalhes de: https://www.nuuvem.com/br-pt/item/resident-evil-6-complete
Raspando detalhes de: https://www.nuuvem.com/br-pt/item/lords-of-the-fallen-goty
Raspando detalhes de: https://www.nuuvem.com/br-pt/item/castlevania-lords-of-shadow-2
Raspando detalhes de: https://www.nuuvem.com/br-pt/item/resident-evil-4-remake
Raspando detalhes de: https://www.nuuvem.com/br-pt/item/silent-hill-f
Raspando detalhes de: https://www.nuuvem.com/br-pt/item/castlevania-lord-of-shadows-mirror-of-fate-hd
Raspando detalhes de: https://www.nuuvem.com/br-pt/item/borderlands-4-super-deluxe

,name,price,url
0,Silent Hill Homecoming,"R$34,99\nR$ 6,99",https://www.nuuvem.com/br-pt/item/silent-hill-...
1,Borderlands 4,"R$379,90\nR$ 341,00",https://www.nuuvem.com/br-pt/item/borderlands-4
2,Castlevania: Lords of Shadow - Ultimate Edition,"R$49,99\nR$ 9,99",https://www.nuuvem.com/br-pt/item/castlevania-...
3,Resident Evil 5 - Gold Edition,"R$89,00\nR$ 22,25",https://www.nuuvem.com/br-pt/item/resident-evi...
4,Digimon Story Time Stranger,"R$319,90\nR$ 287,00",https://www.nuuvem.com/br-pt/item/digimon-stor...


### 3.3. Exercício do Módulo 3

**Tarefa:** Vamos coletar mais informações da Nuuvem. Preço antes do desconto, regiões disponíveis, armazenamento e placa de video mínima e recomendada.

**Objetivo de Aprendizagem:** Este exercício permite que os alunos pratiquem as mesmas habilidades (inspeção, extração, paginação) em um site com HTML mais simples e limpo, reforçando os conceitos centrais sem a complexidade adicional da estrutura de um site comercial.

In [ ]:
# Coloque seu código aqui
all_products_data = []
for link in product_links:
    print(f"Fazendo raspagem de {link}.")
    response = requests.get(link, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    try:
        name = soup.find("h1", {"class":"product-title"}).text.strip()
    except AttributeError:
        name = None
    
    try:
        price_after = soup.find("", {"class":}).text.strip()
    except AttributeError:
        price_after = None

    try:
        storage = soup.find("", {"class":}).text.strip()
    except AttributeError:
        storage = None
    
    try:
        video_card = soup.find("", {"class":}).text.strip()
    except ArithmeticError:
        video_card = None

    product_data = {
        'name': name,
        'price_after': price_after,
        'storage': storage,
        'video_card': video_card
    }
    all_products_data.append(product_data)
    time.sleep(1)

df_jogos = pd.DataFrame(all_products_data)

print(f'\nTotal de produtos raspados: {len(df_jogos)}')
df_jogos.head()

SyntaxError: invalid syntax (1140790599.py, line 9)

---
## Módulo 4: Domando a Web Dinâmica com Playwright

Este módulo aborda as limitações do `requests` ao introduzir a automação de navegador para lidar com sites que renderizam conteúdo usando JavaScript.

### 4.1. Estrutura Conceitual: Navegadores "Headless" e o DOM Renderizado

* **O Problema:** A biblioteca `requests` falha em sites dinâmicos porque só consegue ver o código-fonte HTML inicial retornado pelo servidor. Ela não executa o JavaScript que, em muitos sites modernos, é responsável por buscar dados e construir o conteúdo final da página.
* **A Solução:** Ferramentas como o Playwright resolvem esse problema automatizando um navegador real (como Chromium, Firefox ou WebKit) em segundo plano, em um modo conhecido como "headless" (sem interface gráfica). O Playwright instrui o navegador a carregar a página, aguarda a execução de todo o JavaScript e, em seguida, fornece acesso ao DOM totalmente renderizado e final, exatamente como um usuário o veria.
* **Natureza Assíncrona:** Muitas operações do Playwright são assíncronas por natureza, pois envolvem esperar por eventos de rede lentos. Embora a biblioteca forneça uma API síncrona para simplificar, é útil entender que, por baixo dos panos, ela lida com operações que não são instantâneas.

### 4.2. Pré-requisitos: Instalando o Playwright

A instalação do Playwright é um processo de duas etapas:

1.  Instalar a biblioteca Python: `pip install playwright`
2.  Baixar os binários dos navegadores: `playwright install`

Este segundo comando baixa versões do Chromium, Firefox e WebKit que são garantidas para funcionar com a versão instalada da biblioteca Playwright.

### 4.3. Exemplo: Raspagem de um Site com Rolagem Infinita (Infinite Scroll)

**Objetivo:** Raspar dados de produtos de uma página que carrega continuamente mais itens à medida que o usuário rola para baixo.

In [ ]:
!pip install playwright
!playwright install
!pip3 install nest-asyncio

In [ ]:
import time
import pandas as pd
import asyncio
import nest_asyncio
#from playwright.sync_api import sync_playwright
from playwright.async_api import async_playwright

async def scrape_infinite_scroll(url):
    """Raspa dados de uma página com rolagem infinita."""


    async with async_playwright() as p:
        # 1. Lançar o navegador
        # headless=False permite ver o navegador em ação. Mude para True para rodar em background.
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        # 2. Navegar para a URL
        await page.goto(url, timeout=60000)
        print(f"Navegando para {url}")

        # 3. O Loop de Rolagem
        previous_height = -1
        for _ in range(5): # Limita a 5 rolagens para este exemplo, para não demorar muito
            current_height = await page.evaluate("document.body.scrollHeight")
            if current_height == previous_height:
                print("Fim da rolagem alcançado.")
                break

            previous_height = current_height
            await page.evaluate("window.scrollTo(0, document.body.scrollHeight)")
            print(f"Rolando... Altura atual: {current_height} pixels.")

            # 4. A Espera Crucial
            # Espera para que o novo conteúdo seja carregado via JavaScript
            await page.wait_for_timeout(3000) # 3 segundos de pausa

        # 5. Extração após a rolagem completa
        print("Iniciando extração de dados...")
        # Este seletor é para o site de exemplo 'scrapingcourse.com/infinite-scrolling'
        product_elements = await page.query_selector_all(".product-item")

        print(f"Total de produtos encontrados: {len(product_elements)}\n elemento: {product_elements}")
        scraped_data = []
        for product in product_elements:
            name_element = await product.query_selector(".product-name")
            name = await name_element.inner_text() if name_element else None

            price_element = await product.query_selector(".product-price")
            price = await price_element.inner_text() if price_element else None
            scraped_data.append({'name': name, 'price': price})

        await browser.close()

        # 6. Criação do DataFrame
        return pd.DataFrame(scraped_data)

# URL de exemplo para teste
target_url = "https://scrapingcourse.com/infinite-scrolling/"




# Get the current event loop and run the coroutine
import nest_asyncio
nest_asyncio.apply()
loop = asyncio.get_event_loop()
df_products = loop.run_until_complete(scrape_infinite_scroll(target_url))

# Exibir resultado
print(f"\nTotal de produtos raspados: {len(df_products)}")
df_products.head()

### 4.4. Exercício do Módulo 4

**Tarefa:** Encontre um site de notícias ou e-commerce que utilize um botão "Carregar Mais" ou "Ver Mais" em vez de rolagem infinita. Os alunos devem escrever um script Playwright que:

1.  Navegue até a página.
2.  Entre em um loop que localiza o botão "Carregar Mais" usando seu seletor.
3.  Clique no botão usando `button.click()`.
4.  Aguarde o carregamento do novo conteúdo.
5.  O loop deve continuar até que o botão não esteja mais visível ou esteja desativado.
6.  Finalmente, raspe todos os artigos/produtos carregados para um DataFrame.

**Objetivo de Aprendizagem:** Este exercício ensina um padrão de interação dinâmica diferente, mas relacionado (clicar vs. rolar), e reforça o conceito fundamental de esperar por mudanças de estado em uma página dinâmica.

In [ ]:
# Coloque seu código aqui

---
## Módulo 5: Construindo um Crawler Escalável com Scrapy

O módulo final apresenta o Scrapy, um poderoso framework assíncrono projetado para construir crawlers web robustos e de larga escala.

### 5.1. Estrutura Conceitual: A Arquitetura do Scrapy

* **Scrapy como um Framework:** O Scrapy não é apenas uma biblioteca, mas um framework completo. Ele fornece uma estrutura de projeto e um motor que gerencia todo o processo de rastreamento, desde o agendamento de requisições até o processamento de dados.
* **Motor Assíncrono:** Construído sobre o Twisted, um motor de rede orientado a eventos, o Scrapy pode lidar com muitas requisições concorrentemente (de forma assíncrona). Isso o torna significativamente mais rápido e eficiente em termos de recursos do que um script linear (como os dos módulos anteriores) para tarefas de grande escala.
* **Componentes Principais:**
    * **Spiders:** Classes Python que contêm a lógica de rastreamento para um site específico. Elas definem as URLs iniciais, como seguir links e como analisar as páginas para extrair dados.
    * **Items:** Contêineres de dados estruturados, semelhantes a um dicionário Python com campos pré-definidos. Eles definem o esquema dos dados de saída, garantindo consistência.
    * **Seletores:** O mecanismo embutido do Scrapy para extrair dados usando seletores CSS ou XPath, funcionando de forma semelhante ao BeautifulSoup, mas integrado ao fluxo de resposta.
    * **Pipelines:** Componentes que processam os `Items` após serem raspados. São usados para limpeza de dados, validação, verificação de duplicatas e armazenamento em bancos de dados ou outros formatos.

### 5.2. Exemplo: Rastreando um Site de Notícias em Busca de Artigos

**Objetivo:** Construir um spider que comece na página inicial de um site de notícias (por exemplo, BBC News), encontre links para artigos individuais, siga-os e extraia o título, autor, data de publicação e corpo do texto de cada página de artigo.

#### 1. Configuração do Projeto
Primeiro, vamos criar o projeto e o spider usando a linha de comando do Scrapy. **Execute estes comandos em um terminal separado, não no notebook**, no diretório onde deseja criar o projeto.
```bash
scrapy startproject bbc_scraper
cd bbc_scraper
scrapy genspider bbc_news [bbc.com/news](https://bbc.com/news)
```
Isso cria toda a estrutura de diretórios necessária. Agora, vamos replicar a criação dos arquivos Python usando a "mágica" do Jupyter para que possamos editar e executar tudo a partir daqui.

In [ ]:
# Criando a estrutura de diretórios necessária
import os
os.makedirs('bbc_scraper/spiders', exist_ok=True)

#### 2. Definindo o Item (em `bbc_scraper/items.py`)
Defina a estrutura dos dados que você deseja raspar.

In [ ]:
%%writefile bbc_scraper/items.py
import scrapy

class BbcArticleItem(scrapy.Item):
    url = scrapy.Field()
    title = scrapy.Field()
    author = scrapy.Field()
    publish_date = scrapy.Field()
    body_text = scrapy.Field()

#### 3. Escrevendo o Spider (em `bbc_scraper/spiders/bbc_news.py`)
Esta é a lógica central do crawler. Ele usa um padrão de dois callbacks: um para encontrar os links e outro para extrair os dados.

In [ ]:
%%writefile bbc_scraper/spiders/bbc_news.py
import scrapy
from bbc_scraper.items import BbcArticleItem

class BbcNewsSpider(scrapy.Spider):
    name = 'bbc_news'
    allowed_domains = ['bbc.com']
    start_urls = ['https://www.bbc.com/news']

    def parse(self, response):
        """
        Este método é chamado para a página inicial.
        Sua função é encontrar os links para os artigos e agendar
        sua raspagem usando o callback 'parse_article'.
        """
        # Seletor para encontrar links de artigos (pode precisar de ajuste)
        # Este seletor busca por links que contenham '/news/' e terminem com um número (padrão de artigo da BBC)
        article_links = response.css('a[href*="/news/"]::attr(href)').re(r'.*-\d+$')

        for link in article_links:
            # Usa response.follow para construir URLs absolutas e agendar a requisição
            yield response.follow(link, callback=self.parse_article)

    def parse_article(self, response):
        """
        Este método é chamado para cada página de artigo.
        Sua função é extrair os dados e empacotá-los em um Item.
        """
        # Instancia o item
        item = BbcArticleItem()

        item['url'] = response.url

        # Seletores para extrair os dados (precisam ser verificados com as ferramentas de dev)
        item['title'] = response.css('h1#main-heading::text').get()

        # O autor pode estar em diferentes locais, este é um exemplo
        item['author'] = response.css('p[class*="TextContributorName"] > strong::text').get()

        item['publish_date'] = response.css('time[data-testid="timestamp"]::attr(datetime)').get()

        # O corpo do artigo geralmente é composto por múltiplos parágrafos
        paragraphs = response.css('div[data-component="text-block"] p::text').getall()
        item['body_text'] = ' '.join(paragraphs)

        # Retorna o item preenchido para o motor do Scrapy
        yield item

#### 4. Executando o Crawl e Salvando os Dados
Execute o spider a partir do diretório raiz do projeto. O Scrapy cuidará do resto. Usaremos o `!` para executar o comando do shell a partir do notebook. A flag `-o` utiliza o recurso embutido "Feed Exports", que salva automaticamente todos os `Items` gerados em um arquivo no formato especificado.

In [ ]:
# Executa o crawler a partir do diretório bbc_scraper e salva a saída em articles.jsonl
# O scrapy precisa de um arquivo de configuração, que não criamos, então adicionamos -s LOG_ENABLED=False para simplificar
!cd bbc_scraper && scrapy crawl bbc_news -o articles.jsonl -s LOG_ENABLED=False

#### 5. Carregando no Pandas
Após a conclusão do crawl, o arquivo de saída pode ser facilmente carregado em um DataFrame do pandas.

In [ ]:
import pandas as pd

try:
    # Carrega o arquivo JSON Lines em um DataFrame
    df_articles = pd.read_json('bbc_scraper/articles.jsonl', lines=True)
    print(f"Total de artigos raspados: {len(df_articles)}")
    print(df_articles.head())
except FileNotFoundError:
    print("Arquivo articles.jsonl não encontrado. Certifique-se de que o crawler foi executado com sucesso.")

### 5.3. Exercício do Módulo 5

**Tarefa:** Modifiquem seu spider de notícias. Em vez de começar na página inicial, eles devem começar em uma página de categoria específica (por exemplo, a seção "Technology" ou "Business" do site de notícias). Eles precisarão adaptar seus seletores de busca de links para funcionar na nova estrutura da página e rastrear todos os artigos dentro daquela categoria.

**Objetivo de Aprendizagem:** Isso ensina os alunos a adaptar um spider a diferentes layouts de página dentro do mesmo site e reforça a importância da fase de inspeção para a criação de scrapers robustos.

In [ ]:
# Coloque seu código aqui

---

## Conclusão: Escolhendo sua Ferramenta e Raspando com Ética

Este guia percorreu o espectro da aquisição de dados, desde a interação ordenada com APIs até a construção de crawlers complexos e assíncronos. A jornada de cinco módulos demonstrou que cada metodologia possui um conjunto único de pontos fortes e fracos, tornando a seleção da ferramenta uma decisão estratégica baseada na arquitetura do alvo. APIs oferecem confiabilidade ao custo de flexibilidade. `Requests` e `BeautifulSoup` fornecem uma entrada simples para a raspagem de sites estáticos. `Playwright` preenche a lacuna para a web moderna e dinâmica, enquanto o `Scrapy` oferece a potência necessária para tarefas de coleta de dados em larga escala.

No entanto, a habilidade técnica deve ser sempre acompanhada de responsabilidade ética. Como coletores de dados, é imperativo operar de maneira a respeitar os recursos e as regras dos provedores de dados. Antes de iniciar qualquer projeto de scraping, siga estas diretrizes fundamentais:

* **Consulte o `robots.txt`:** Sempre verifique o arquivo `robots.txt` de um site (geralmente em `www.exemplo.com/robots.txt`). Ele especifica quais partes do site os proprietários não desejam que sejam acessadas por robôs. Respeitar este arquivo é a primeira regra da raspagem ética.
* **Raspe de Forma Responsável:** Evite sobrecarregar os servidores de um site. Faça requisições em um ritmo razoável. Ferramentas como o Scrapy facilitam isso com configurações como `DOWNLOAD_DELAY`. Para scripts mais simples, a inserção manual de pausas (`time.sleep()`) entre as requisições é uma prática recomendada.
* **Respeite os Termos de Serviço:** Os Termos de Serviço de um site geralmente descrevem as regras de uso, que podem incluir cláusulas sobre raspagem de dados automatizada. Esteja ciente dessas regras.
* **Identifique-se:** Use um `User-Agent` descritivo que identifique seu bot. Isso permite que os administradores do site entrem em contato com você caso seu scraper esteja causando problemas.

Ao combinar proficiência técnica com uma abordagem ética, a aquisição de dados se torna uma ferramenta poderosa e sustentável para análise, pesquisa e inovação.